In [1]:
import string
import numpy as np
import matplotlib.pyplot as plt
import os
from prettytable import PrettyTable
from IPython.display import Latex


import time
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from copy import deepcopy
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [8]:
seed = 99
np.random.seed(seed)

mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# valid_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std),
# ])

train_set = torchvision.datasets.CIFAR10(
    root='./data/',
    train=True,
    transform=train_transform,
    download=True,
)

# valid_set = torchvision.datasets.CIFAR10(
#     root='./data/',
#     train=True,
#     transform=valid_transform,
#     download=True,
# )

# num_train = len(train_set)
# indices = list(range(num_train))
# split = int(np.floor(0.2 * num_train))

# train_idx, valid_idx = indices[split:], indices[:split]
# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

# y_train = np.array(train_set.targets[split:])
# y_valid = np.array(train_set.targets[:split])
y_train = np.array(train_set.targets)

Files already downloaded and verified


In [11]:
for positive_threshold in [0.7, 0.8, 0.85, 0.9]:
    for add_criterion in [14, 16, 17, 18]:
        for learning_rate in [0.01]:
            dirs = 'saved_precision/positive_threshold_%.2f_add_criterion_%d_learning_rate_%.3f_without_validation'\
            %(positive_threshold, add_criterion, learning_rate)
            if not os.path.exists(dirs):
                continue
            print('positive threshold: ', positive_threshold)
            print('add criterion: ', add_criterion)
            print('learning rate: ', learning_rate)
#             validation_precision_matrix = [['/' for _ in range(10)] for _ in range(10)]
#             validation_number_matrix = [['/' for _ in range(10)] for _ in range(10)]
            training_precision_matrix = [['/' for _ in range(10)] for _ in range(10)]
            training_number_matrix = [['/' for _ in range(10)] for _ in range(10)]
            for label in range(10):
                try:
#                     validation_file = open(dirs + '/validation_label%d.txt'%label)
                    training_file = open(dirs + '/training_label%d.txt'%label)
                    lines = training_file.readlines()
                    training_precision = '['
                    for line in lines:
                        training_precision += line.replace('\n','').replace(' ', ',')
                    training_precision += ']'
                    training_precision = eval(training_precision.replace('][','], [').replace(',,,,', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '['))
                    true_positive_index = list(np.where(y_train == label)[0])
                    for k in range(len(training_precision)):
                        index = training_precision[k]
                        TP = len(list(set(index) & set(true_positive_index)))
                        if len(index) > 0:
                            training_precision_matrix[k][label] = np.round(TP/len(index), 3)
                        else:
                            training_precision_matrix[k][label] = 0
                        training_number_matrix[k][label] = len(index)
                    lines = validation_file.readlines()
#                     validation_precision = '['
#                     for line in lines:
#                         validation_precision += line.replace('\n','').replace(' ', ',')
#                     validation_precision += ']'
#                     validation_precision = eval(validation_precision.replace('][','], [').replace(',,,', ',').replace(',,', ',').replace('[,', '['))
#                     true_positive_index = list(np.where(y_valid == label)[0])
#                     for k in range(len(validation_precision)):
#                         index = validation_precision[k]
#                         TP = len(list(set(index) & set(true_positive_index)))
#                         if len(index) > 0:
#                             validation_precision_matrix[k][label] = np.round(TP/len(index), 3)
#                         else:
#                             validation_precision_matrix[k][label] = 0
#                         validation_number_matrix[k][label] = len(index)
                except:
                    pass
#             file = open('additional_data_threshold_'+str(positive_threshold)+'_criterion_'+str(add_criterion)+'.txt', 'w')
#             print('\nResult on validation set')   
# #             file.write('Result on validation set\n')
#             table = PrettyTable(['k', 'airplane','auto','bird','cat','deer','dog','frog','horse','ship','truck'])
#             for k in range(10):
#                 table.add_row([k]+[str(validation_precision_matrix[k][i])+'('+str(validation_number_matrix[k][i])+')' for i in range(10)])
#             print(table)
#             file.write(str(table))
            print('Result on training set')  
#             file.write('\nResult on training set\n')
            table = PrettyTable(['k', 'airplane','auto','bird','cat','deer','dog','frog','horse','ship','truck'])
            for k in range(10):
                table.add_row([k]+[str(training_precision_matrix[k][i])+'('+str(training_number_matrix[k][i])+')' for i in range(10)])
            print(table, '\n')
#             file.write(str(table))
#             file.close()


positive threshold:  0.7
add criterion:  14
learning rate:  0.01
Result on training set
+---+----------+------+------+------+------+------+------+-------+------+-------+
| k | airplane | auto | bird | cat  | deer | dog  | frog | horse | ship | truck |
+---+----------+------+------+------+------+------+------+-------+------+-------+
| 0 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 1 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 2 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 3 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 4 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 5 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 6 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 7 |   /(/)   | /(/) | /(/) | /(/) | /(/) | /(/) | /(/) |  /(/) | /(/) |  /(/) |
| 8 |   /(